In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI4Good/Dataset_June_8.csv')

In [ ]:
# pretending that k=10
temp = df.sample(10)

temp

,Unnamed: 0,Latitude,Longitude,OFFENCE,DAYS_SINCE
261990,300995,43.759817,-79.225321,Robbery,3464.0
246731,283570,43.738148,-79.529922,Break and Enter,3305.0
120922,139038,43.668202,-79.333620,Auto Theft,1877.0
1281,1460,43.642105,-79.404749,Assault,384.0
233277,268155,43.677556,-79.313712,Assault,3154.0
44561,50968,43.733929,-79.450319,Auto Theft,944.0
52747,60289,43.710906,-79.399285,Theft Over,1039.0
259138,297698,43.688441,-79.300543,Robbery,3436.0
44324,50707,43.672771,-79.321360,Assault,941.0
72607,83164,43.711727,-79.373897,Break and Enter,1291.0


In [ ]:
dict_offense = {'Assault': 9, 'Break and Enter': 1, 'Robbery': 5, 'Auto Theft': 1, 'Homicide': 15, 'Theft Over': 10}

temp2=temp.replace({"OFFENCE": dict_offense})

temp2

,Unnamed: 0,Latitude,Longitude,OFFENCE,DAYS_SINCE
261990,300995,43.759817,-79.225321,5,3464.0
246731,283570,43.738148,-79.529922,1,3305.0
120922,139038,43.668202,-79.333620,1,1877.0
1281,1460,43.642105,-79.404749,9,384.0
233277,268155,43.677556,-79.313712,9,3154.0
44561,50968,43.733929,-79.450319,1,944.0
52747,60289,43.710906,-79.399285,10,1039.0
259138,297698,43.688441,-79.300543,5,3436.0
44324,50707,43.672771,-79.321360,9,941.0
72607,83164,43.711727,-79.373897,1,1291.0


In [ ]:
# helper function to determine how far away a point is to a centroid

centroid = [43.7598175487073,	-79.2253215487073]

def centroid_distance(lat, long, centroid):
  c = math.sqrt((centroid[0] - lat)**2 + (centroid[1] - long)**2)
  return c

centroid_distance(temp.iloc[0]['Latitude'], temp.iloc[0]['Longitude'], centroid)

#print(temp.iloc[0]['Latitude'])
#print(centroid[0])
#print((temp.iloc[0]['Latitude'] - centroid[0])**2)
#print((temp.iloc[0]['Longitude'] - centroid[1])**2)

1.1054265259999559e-06

In [ ]:
# helper function to create weights for distence
# outputs a multiplier

weight1 = 1
weight2 = 0.9
weight3 = 0.75
weight4 = 0.6
#weight5 = 0.4
#weight6 = 0.2


def distance_weight(distance):
  if distance <= 0.000001:                #
    return weight1
  elif 0.000001 < distance <= 0.000002:   #
    return weight2
  elif 0.000002 < distance <= 0.000003:   #
    return weight3
  elif 0.000003 < distance:               #
    return weight4
#  elif __ < distance <= ___:         #
#    return weight5
#  elif ___ < distance:                 #
#    return weight6

distance_weight(centroid_distance(temp.iloc[0]['Latitude'], temp.iloc[0]['Longitude'], centroid))

0.9

In [ ]:
# helper function to create weights for days_since
# outputs a multiplier

weight1 = 1
weight2 = 0.9
weight3 = 0.75
weight4 = 0.6
weight5 = 0.4
weight6 = 0.2


def day_weight(days):
  if days <= 30:                  #<1 month
    return weight1
  elif 30 < days <= 180:          #1-3 months
    return weight2
  elif 180 < days <= 365:         #3-12 months
    return weight3
  elif 365 < days <= 2*365:       #1-2 years
    return weight4
  elif 2*365 < days <= 5*365:     #2-5 years
    return weight5
  elif 5*365 < days:              #5-10 years
    return weight6

day_weight(366)
# day_weight(df.iloc[0]["OFFENCE"], df.iloc[0]["DAYS_SINCE"])

0.6

In [ ]:
# to take in k instances and give total safety score

def safety_score(centroid, df):
  '''
  pd.dataframe -> Int

  Takes in a dataframe. For each row, calculates the distance to the centroid,
  assigns a weight for the distance and the days_since, applies weight to the
  offence score, then sums up all the scores

  '''
  scores = []
  for i in range(len(df)):
    lat = df.iloc[i]['Latitude']
    long = df.iloc[i]['Longitude']
    offence = df.iloc[i]['OFFENCE']
    days_since = df.iloc[i]['DAYS_SINCE']

    distance = centroid_distance(lat, long, centroid)           #calc distance
    distance_adj = distance * distance_weight(distance)         #adjust to distance weight
    day_adj = distance_adj * day_weight(days_since)             #adjust to days weight
    offence_adj = day_adj * offence
    scores.append(offence_adj)

  return sum(scores)

c = [43.7598175487073,	-79.2253215487073]
safety_score(c, temp2)

1.747081337802221

Notes from Yanan:
places w higher occurences should have higher scores too - try w june_15 notes
try to change c with the top lat/longs from june_15
validate my function. if higher close, but longer ago. should be more safe